In [1]:
import pandas as pd
import json
from datetime import datetime
from pprint import pprint

In [2]:
ca_videos = pd.read_csv("Trending-YouTube-Data-Viz/youtube-new/CAvideos.csv")
de_videos = pd.read_csv("Trending-YouTube-Data-Viz/youtube-new/DEvideos.csv")
fr_videos = pd.read_csv("Trending-YouTube-Data-Viz/youtube-new/FRvideos.csv")
gb_videos = pd.read_csv("Trending-YouTube-Data-Viz/youtube-new/GBvideos.csv")
us_videos = pd.read_csv("Trending-YouTube-Data-Viz/youtube-new/USvideos.csv")

IOError: File Trending-YouTube-Data-Viz/youtube-new/CAvideos.csv does not exist

In [3]:
ca_videos.columns

NameError: name 'ca_videos' is not defined

In [4]:
len(ca_videos.trending_date.unique()) # YY-DD-MM

145

In [5]:
ca_videos.shape

(28919, 16)

In [6]:
def category_dict(file):
    category_dict = {}
    with open(file) as json_data:
        d = json.load(json_data)
        for item in d["items"]:
            category_dict[int(item["id"])] = item["snippet"]["title"]
    category_dict[29] = "Nonprofits & Activism"
    return category_dict

In [7]:
us_category = category_dict("Trending-YouTube-Data-Viz/youtube-new/US_category_id.json")

In [8]:
de_category = category_dict("Trending-YouTube-Data-Viz/youtube-new/DE_category_id.json")

In [9]:
ca_category = category_dict("Trending-YouTube-Data-Viz/youtube-new/CA_category_id.json")

In [10]:
gb_category = category_dict("Trending-YouTube-Data-Viz/youtube-new/GB_category_id.json")

In [11]:
fr_category = category_dict("Trending-YouTube-Data-Viz/youtube-new/FR_category_id.json")

In [12]:
ca_videos["category"] = ca_videos["category_id"].map(ca_category)

In [13]:
us_videos["category"] = us_videos["category_id"].map(us_category)

In [14]:
de_videos["category"] = de_videos["category_id"].map(de_category)

In [15]:
gb_videos["category"] = gb_videos["category_id"].map(gb_category)
fr_videos["category"] = fr_videos["category_id"].map(fr_category)

In [16]:
gb_videos["trending_date"] = gb_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [17]:
fr_videos = fr_videos.drop([18054, 18055], axis=0)

In [18]:
us_videos["trending_date"] = us_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))
ca_videos["trending_date"] = ca_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [19]:
fr_videos["trending_date"] = fr_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))
de_videos["trending_date"] = de_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [20]:
gb_videos["trending_month"] = gb_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [21]:
us_videos["trending_month"] = us_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [22]:
ca_videos["trending_month"] = ca_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [23]:
fr_videos["trending_month"] = fr_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [24]:
de_videos["trending_month"] = de_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

### IF WE REMOVE ALL THE VIDEOS WITH 0 LIKES AND 0 COMMENTS

In [89]:
us_videos = us_videos[(us_videos.comment_count != 0) & (us_videos.likes != 0)]

#### dataframe for category counts

In [25]:
# def count_cat(df):
#     cat_count_df = df \
#         .groupby(["trending_month", "category"])["category_id"].count().reset_index(name="count")
#     return cat_count_df
def count_cat(df):
    cat_count_df = df \
        .groupby(["trending_month", "category"])\
        .agg({"category_id": "count", "comment_count": "sum", "likes": "sum"})[["category_id", "comment_count", "likes"]]\
        .reset_index()
    cat_count_df.columns = ["trending_month", "category", "cat_cnt", "sum_comment_cnt", "sum_likes_cnt"]
    return cat_count_df

In [91]:
gb_mntly = count_cat(gb_videos)
gb_mntly["country"] = "Great Britain"

## --------------------- ##

In [26]:
us_mntly = count_cat(us_videos)
us_mntly["country"] = "United States"

## --------------------- ##

In [27]:
ca_mntly = count_cat(ca_videos)
ca_mntly["country"] = "Canada"

In [28]:
de_mntly = count_cat(de_videos)
de_mntly["country"] = "Germany"

In [95]:
fr_videos["likes"] = fr_videos["likes"].apply(int)

In [96]:
fr_mntly = count_cat(fr_videos)
fr_mntly["country"] = "France"

In [97]:
gb_videos.to_csv("Trending-YouTube-Data-Viz/gb_videos.csv", index=False)

In [98]:
fr_videos.to_csv("Trending-YouTube-Data-Viz/fr_videos.csv", index=False)

In [99]:
ca_videos.to_csv("Trending-YouTube-Data-Viz/ca_videos.csv", index=False)

In [100]:
us_videos.to_csv("Trending-YouTube-Data-Viz/us_videos.csv", index=False)

In [101]:
de_videos.to_csv("Trending-YouTube-Data-Viz/de_videos.csv", index=False)

## ---------------------------------

In [27]:
us_mntly = us_mntly.drop([9], axis=0)

In [56]:
us_mntly["sum_likes_cnt"] = round(us_mntly["sum_likes_cnt"] / 1000, 3)

In [28]:
us_mntly.to_csv("Trending-YouTube-Data-Viz/us_mntly.csv", index=False)

In [4]:
us_mntly.sum_comment_cnt.describe()

count    9.000000e+01
mean     1.991316e+06
std      3.112370e+06
min      1.507000e+03
25%      2.149242e+05
50%      8.766205e+05
75%      2.383412e+06
max      1.588929e+07
Name: sum_comment_cnt, dtype: float64

In [5]:
us_mntly.sum_likes_cnt.describe()

count    9.000000e+01
mean     1.624352e+07
std      2.747232e+07
min      8.382000e+03
25%      1.397635e+06
50%      6.433709e+06
75%      1.572878e+07
max      1.622845e+08
Name: sum_likes_cnt, dtype: float64

### combine more data

In [35]:
all_mntly = gb_mntly.append(fr_mntly).append(ca_mntly).append(us_mntly).append(de_mntly)

NameError: name 'gb_mntly' is not defined

In [54]:
all_mntly.to_csv("Trending-YouTube-Data-Viz/all_mntly.csv", index=False)

### Plotly (us_mntly)

In [1]:
from plotly.offline import init_notebook_mode, iplot

import plotly
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

init_notebook_mode(connected=True) 

In [2]:
import time
import pandas as pd

In [40]:
us_mntly = pd.read_csv("Trending-YouTube-Data-Viz/us_mntly.csv")

In [4]:
# us_mntly["sum_comment_cnt"] = round((us_mntly["sum_comment_cnt"] / 1000), 3)

In [5]:
# us_mntly["sum_likes_cnt"] = round(us_mntly["sum_likes_cnt"] / 1000, 3)

In [6]:
us_mntly.sum_comment_cnt.describe()

count    9.000000e+01
mean     1.991316e+06
std      3.112370e+06
min      1.507000e+03
25%      2.149242e+05
50%      8.766205e+05
75%      2.383412e+06
max      1.588929e+07
Name: sum_comment_cnt, dtype: float64

In [7]:
us_mntly.sum_likes_cnt.describe()

count    9.000000e+01
mean     1.624352e+07
std      2.747232e+07
min      8.382000e+03
25%      1.397635e+06
50%      6.433709e+06
75%      1.572878e+07
max      1.622845e+08
Name: sum_likes_cnt, dtype: float64

In [8]:
plotly.tools.set_credentials_file(username='jingjuewang', api_key='Jlw5mC8COv1lXpqTUyMP')

In [9]:
table = FF.create_table(us_mntly.head(10))

/Users/jingjuewang/anaconda/envs/fastai/lib/python3.6/site-packages/plotly/tools.py:1491: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table



In [10]:
iplot(table)

In [11]:
month_from_col = set(us_mntly["trending_month"])

In [12]:
month_list = sorted(list(month_from_col))

In [13]:
# make list of categories
categories = []
for category in us_mntly["category"]:
    if category not in categories:
        categories.append(category)

In [14]:
columns = []

In [15]:
# make grid
for month in month_list:
    for cat in categories:
        dataset_by_month = us_mntly[us_mntly["trending_month"] == month]
        dataset_by_month_and_cat = dataset_by_month[dataset_by_month["category"] == cat]
        for col_name in dataset_by_month_and_cat:
            column_name = '{month}_{cat}_{header}_gapminder_grid'.format(
                month=month, cat=cat, header=col_name
            )
            a_column = Column(list(dataset_by_month_and_cat[col_name]), column_name)
            columns.append(a_column)

In [16]:
# upload grid 
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'gapminder_grid'+str(time.time()), auto_open=False)

In [17]:
url

'https://plot.ly/~jingjuewang/74/'

### make the figure

In [18]:
figure = {
    'data': [],
    "layout": {},
    "frames": [],
    "config": {'scrollzoom': True}
}

In [19]:
# fill in most of layout
# figure['layout']['xaxis'] = {'range': [1, 20000], 'title': 'total comment count', 'gridcolor': '#FFFFFF'}
# figure['layout']['yaxis'] = {'range': [8, 200000], 'title': 'total likes count', 'gridcolor': '#FFFFFF'}
# figure['layout']['hovermode'] = 'closest'
# figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'


In [20]:
x_min = us_mntly.sum_comment_cnt.min()
x_max = us_mntly.sum_comment_cnt.max()
y_min = us_mntly.sum_likes_cnt.min()
y_max = us_mntly.sum_likes_cnt.max()

In [21]:
# fill in most of layout
figure['layout']['xaxis'] = {'range': [x_min, x_max], "autorange": "true",'title': 'total comment count', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'range': [y_min, y_max], "autorange": "true",'title': 'total likes count', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = '#FFFFFF'

### Add slider

In [22]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'month:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

### Add Play and Pause Buttons

In [23]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = {
    'Autos & Vehicles': 'rgb(171, 99, 250)',
    'Comedy': 'rgb(230, 99, 250)',
    'Education': 'rgb(99, 110, 250)',
    'Entertainment': 'rgb(25, 211, 243)',
    'Film & Animation': 'rgb(50, 170, 255)',
    'Gaming': 'rgb(141,211,199)',
    'Howto & Style': 'rgb(255,255,179)',
    'Music': 'rgb(190,186,218)',
    'News & Politics': 'rgb(251,128,114)',
    'People & Blogs': 'rgb(128,177,211)',
    'Pets & Animals': 'rgb(253,180,98)',
    'Science & Technology': 'rgb(179,222,105)',
    'Sports': 'rgb(252,205,229)',
    'Travel & Events': 'rgb(217,217,217)',
    'Nonprofits & Activism': 'rgb(188,128,189)',
    'Shows': 'rgb(204,235,197)',
}

In [24]:
us_mntly.category.unique()

array(['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment',
       'Film & Animation', 'Gaming', 'Howto & Style', 'Music',
       'News & Politics', 'People & Blogs', 'Pets & Animals',
       'Science & Technology', 'Sports', 'Travel & Events',
       'Nonprofits & Activism', 'Shows'], dtype=object)

#### Fill in Figure with Data and Frames

In [25]:
col_name_template = '{month}_{cat}_{header}_gapminder_grid'
month = "17-11"
for cat in categories:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='sum_comment_cnt'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='sum_likes_cnt'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='country'
        )),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 month=month, cat=cat, header='cat_cnt'
            )),
            'color': custom_colors[cat]
        },
        'name': cat
    }
    figure['data'].append(data_dict)

### create frames

In [26]:
for month in month_list:
    frame = {'data': [], 'name': str(month)}
    for cat in categories:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='sum_comment_cnt'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='sum_likes_cnt'
            )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='country'
                )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                    month=month, cat=cat, header='cat_cnt'
                )),
                'color': custom_colors[cat],
                "opacity": 0.8
            },
            'name': cat
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [month],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': month,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [27]:
py.icreate_animations(figure, 'gapminder_example'+str(time.time()))

# OFFLINE

In [28]:
import pandas as pd
import numpy as np
from __future__ import division
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [41]:
df = pd.DataFrame()
for month in month_list:
    for cat in categories:
        dataset_by_month = us_mntly[us_mntly["trending_month"] == month]
        dataset_by_month_and_cat = dataset_by_month[dataset_by_month["category"] == cat]
        for col_name in dataset_by_month_and_cat:
            temp = '{month}_{cat}_{header}_gapminder_grid'.format(
                month=month, cat=cat, header=col_name
            )
            df = df.append({'value': list(dataset_by_month_and_cat[col_name]), 'key': temp}, ignore_index=True)


In [42]:
figure = {
    'data': [],
    "layout": {},
    "frames": [],
    "config": {'scrollzoom': True}
}

In [43]:
x_min = us_mntly.sum_comment_cnt.min()
x_max = us_mntly.sum_comment_cnt.max()
y_min = us_mntly.sum_likes_cnt.min()
y_max = us_mntly.sum_likes_cnt.max()

In [44]:
# fill in most of layout
figure['layout']['xaxis'] = {'range': [x_min, x_max], "autorange": "true",'title': 'total comment count', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'range': [y_min, y_max], "autorange": "true",'title': 'total likes count', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = '#FFFFFF'

In [45]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'month:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [46]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = {
    'Autos & Vehicles': 'rgb(171, 99, 250)',
    'Comedy': 'rgb(230, 99, 250)',
    'Education': 'rgb(99, 110, 250)',
    'Entertainment': 'rgb(25, 211, 243)',
    'Film & Animation': 'rgb(50, 170, 255)',
    'Gaming': 'rgb(141,211,199)',
    'Howto & Style': 'rgb(255,255,179)',
    'Music': 'rgb(190,186,218)',
    'News & Politics': 'rgb(251,128,114)',
    'People & Blogs': 'rgb(128,177,211)',
    'Pets & Animals': 'rgb(253,180,98)',
    'Science & Technology': 'rgb(179,222,105)',
    'Sports': 'rgb(252,205,229)',
    'Travel & Events': 'rgb(217,217,217)',
    'Nonprofits & Activism': 'rgb(188,128,189)',
    'Shows': 'rgb(204,235,197)',
}

In [47]:
col_name_template = '{month}_{cat}_{header}_gapminder_grid'
month = "17-11"
for cat in categories:
    data_dict = {
        'xsrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='sum_comment_cnt'
        ), 'value'].values[0],
        'ysrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='sum_likes_cnt'
        ), 'value'].values[0],
        'mode': 'markers',
        'textsrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='country'
        ), 'value'].values[0],
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'sizesrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='cat_cnt'
        ), 'value'].values[0],
            'color': custom_colors[cat]
        },
        'name': cat
    }
    figure['data'].append(data_dict)

In [48]:
for month in month_list:
    frame = {'data': [], 'name': str(month)}
    for cat in categories:
        data_dict = {
            'xsrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='sum_comment_cnt'
        ), 'value'].values[0],
        'ysrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='sum_likes_cnt'
        ), 'value'].values[0],
        'mode': 'markers',
        'textsrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='country'
        ), 'value'].values[0],
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'sizesrc': df.loc[df['key']==col_name_template.format(
            month=month, cat=cat, header='cat_cnt'
        ), 'value'].values[0],
                'color': custom_colors[cat],
                "opacity": 0.8
            },
            'name': cat
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [month],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': month,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [49]:
iplot(figure, config={"scrollzoom": True}, validate=False)